In [1]:
%pip install bitshuffle
%pip install Pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import blimpy as bl
#from ultralytics import YOLO
import time
import os
import random
import psutil
import scipy.ndimage
from PIL import Image
from PIL import ImageDraw
from pathlib import Path
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
import cv2
from scipy.ndimage import label as connected_components
from skimage.filters import sobel
from skimage.segmentation import slic
from skimage.util import img_as_float
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from random import shuffle
from tqdm import tqdm
from skimage.util import img_as_float
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from skimage.segmentation import slic
import scipy.ndimage as ndi
%matplotlib inline

In [4]:
df = pd.read_csv('/home/cgchoza/galaxies/complete_cadences_catalog.csv')
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
36553,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36554,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36555,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36556,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [6]:
df['.h5 path'] = df['.h5 path'].str.replace('0000.h5', '0002.h5', regex=False)
df = df.drop_duplicates(subset='.h5 path', keep='first').reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
30309,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30310,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30311,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30312,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [7]:
df = df[~df['.h5 path'].str.contains('spliced')].reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
1,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
2,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
3,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
4,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
...,...,...,...,...,...,...,...,...
29341,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29342,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29343,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29344,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
df = df.drop(index = 17546)
import pandas as pd
import blimpy as bl
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

def scan_file_for_band(
    h5_path: str,
    band: str,
    fmin: float,
    fmax: float,
    default_nfpc: int = 1024
):
    """
    Open one .h5, read header, return list of dicts for all channels
    whose freq range lies in [fmin, fmax].
    """
    records = []
    fb = bl.Waterfall(h5_path, load_data=False)
    hdr = fb.header

    # skip if not the right band
    # you could also pass row['Band'] in here if you like
    # if row['Band'] != band: return []

    fch1   = hdr['fch1']
    foff   = hdr['foff']
    nchans = hdr.get('nchans')
    nfpc   = hdr.get('nfpc', default_nfpc)
    n_coarse = nchans // nfpc

    for ch in range(n_coarse):
        f0 = ch * nfpc
        f1 = (ch+1) * nfpc
        f_start = fch1 + f0 * foff
        f_stop  = fch1 + (f1-1) * foff
        if (f_start <= fmax) and (f_stop >= fmin):
            records.append({
                '.h5 path': h5_path,
                'channel':    ch,
                'Band':       band,
                'f_start':    f_start,
                'f_stop':     f_stop
            })

    return records

def parallel_filter_df(
    df: pd.DataFrame,
    band: str = "L",
    fmin: float = 1500,
    fmax: float = 1650,
    max_workers: int = 8
) -> pd.DataFrame:
    # restrict to Band==L first to cut the task list down
    df_band = df[df['Band'] == band]
    paths   = df_band[".h5 path"].unique().tolist()

    all_records = []
    with ProcessPoolExecutor(max_workers=max_workers) as exe:
        futures = {
            exe.submit(scan_file_for_band, p, band, fmin, fmax): p
            for p in paths
        }
        for fut in tqdm(as_completed(futures), total=len(futures), ascii=True,
                        desc=f"Scanning {band}-band files"):
            try:
                recs = fut.result()
                all_records.extend(recs)
            except Exception as e:
                h5 = futures[fut]
                print(f"Error on {h5}: {e!r}")

    return pd.DataFrame.from_records(all_records)

# Usage:
df = parallel_filter_df(df, band="L", fmin=1500, fmax=1650, max_workers=2)
print(f"Kept {len(df)} channel entries in L-band 1500-1650 MHz")
# # 1) sample, keeping the original index labels
# df_old = df.sample(n=1000, random_state=42)
# # 2) drop those exact rows from df
# df = df.drop(index=df_old.index).reset_index(drop=True)
# # 4) finally, sample 10,000 more from the remainder
old_df = df.sample(n=10000, random_state=42)
df = df.drop(index=old_df.index).reset_index(drop=True)
df = df.sample(n=1000, random_state=42).reset_index(drop=True)


In [ ]:
# fb = bl.Waterfall(df['.h5 path'].iloc[-1], load_data=True)
# fb.info()


--- File Info ---
DIMENSION_LABELS : [b'time' b'feed_id' b'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :               10501.46484375 MHz
            foff :         -0.00286102294921875 MHz
           ibeam :                               -1
      machine_id :                               20
          nbeams :                                1
           nbits :                               32
          nchans :                            65536
            nifs :                                1
     rawdatafile : guppi_59411_54386_HIP30264_0094.0000.raw
     source_name :                         HIP30264
         src_dej :                     -8:26:53.228
         src_raj :                      6:21:58.451
    telescope_id :                                6
           tsamp :                1.073741823999999
   tstart (ISOT) :          2021-07-16T15:06:26.000
    tstart (MJD) :                59

In [ ]:
import psutil
import time

def ensure_cpu_mem(bytes_needed, safety=0.8):
    avail = psutil.virtual_memory().available
    if bytes_needed > avail * safety:
        time.sleep(120)

def ensure_gpu_mem(bytes_needed, safety=0.8):
    free, total = cp.cuda.runtime.memGetInfo()
    if bytes_needed > free * safety:
        time.sleep(120)


In [ ]:
def yolo_iou(boxA, boxB):
    """
    Compute IoU between two YOLO-format boxes (cx, cy, w, h), normalized [0..1].
    """
    cxA, cyA, wA, hA = boxA
    cxB, cyB, wB, hB = boxB

    # convert to corner coords
    x0A, y0A = cxA - wA/2, cyA - hA/2
    x1A, y1A = cxA + wA/2, cyA + hA/2
    x0B, y0B = cxB - wB/2, cyB - hB/2
    x1B, y1B = cxB + wB/2, cyB + hB/2

    # intersection
    xi0, yi0 = max(x0A, x0B), max(y0A, y0B)
    xi1, yi1 = min(x1A, x1B), min(y1A, y1B)
    inter_w  = max(0.0, xi1 - xi0)
    inter_h  = max(0.0, yi1 - yi0)
    inter    = inter_w * inter_h

    # union
    areaA = wA * hA
    areaB = wB * hB
    union = areaA + areaB - inter
    if union <= 0:
        return 0.0
    return inter / union


def merge_overlaps(boxes, iou_thresh=0.3):
    """
    Group boxes that overlap above `iou_thresh`, then merge each group
    into the tightest bounding rectangle.
    Input:
      boxes: List of (cx, cy, w, h)
    Returns:
      merged: List of (cx, cy, w, h)
    """
    clusters = []
    used = set()

    for i, b in enumerate(boxes):
        if i in used:
            continue
        # start a new cluster with box i
        cluster = [b]
        used.add(i)

        # greedily absorb any box that overlaps any member of the cluster
        changed = True
        while changed:
            changed = False
            for j, bj in enumerate(boxes):
                if j in used:
                    continue
                if any(yolo_iou(bj, ck) > iou_thresh for ck in cluster):
                    cluster.append(bj)
                    used.add(j)
                    changed = True

        clusters.append(cluster)

    # now merge each cluster into one box
    merged = []
    for cl in clusters:
        # convert each to (x1,y1,x2,y2)
        rects = []
        for cx, cy, w, h in cl:
            x1, y1 = cx - w/2, cy - h/2
            x2, y2 = cx + w/2, cy + h/2
            rects.append((x1, y1, x2, y2))

        x1 = min(r[0] for r in rects)
        y1 = min(r[1] for r in rects)
        x2 = max(r[2] for r in rects)
        y2 = max(r[3] for r in rects)

        # back to (cx,cy,w,h)
        merged.append(((x1 + x2)/2, (y1 + y2)/2, x2 - x1, y2 - y1))

    return merged



In [ ]:
import cv2
import numpy as np

def generate_yolo_boxes(binary_mask,
                        min_area=100,
                        max_frac_area=1.0,
                        min_aspect=0.1,
                        max_aspect=10.0):
    """
    Extract YOLO-style boxes from a binary mask via contours.

    Args:
      binary_mask: 2D uint8 array of 0/1
      min_area: drop components smaller than this many pixels
      max_frac_area: drop components larger than this fraction of image area
      min_aspect, max_aspect: drop if w/h not in [min_aspect,max_aspect]
    Returns:
      List of (cx, cy, w, h) in normalized coords [0..1]
    """
    H, W = binary_mask.shape
    total = H * W

    # must be uint8 0/255 for findContours
    mask8 = (binary_mask.astype(np.uint8) * 255)

    # find external contours
    contours, _ = cv2.findContours(mask8,
                                   cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE)

    boxes = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        area = w * h
        # area filters
        if area < min_area or area > max_frac_area * total:
            continue

        ar = w / float(h) if h>0 else 0
        if ar < min_aspect or ar > max_aspect:
            continue

        # center & size in normalized coords
        cx = (x + w/2) / W
        cy = (y + h/2) / H
        boxes.append((cx, cy, w / W, h / H))

    return boxes


In [ ]:
def build_tasks(df):
    """
    Build a flat list of (h5_path, channel_idx) from your DataFrame,
    skipping any channels that fall into known notch filter frequency ranges.
    """
    GBT_NOTCH_FILTERS = {
        "L": [(1200, 1340)],
        "S": [(2300, 2360)],
    }

    tasks = []

    for _, row in df.iterrows():
        h5 = row[".h5 path"]
        band = row["Band"]  # e.g., 'L', 'S', etc.
        fb = bl.Waterfall(h5, load_data=False)
        nfreq = fb.header.get("nchans")
        nfpc = fb.header.get("nfpc", 1024)
        fch1 = fb.header["fch1"]
        foff = fb.header["foff"]
        n_coarse = nfreq // nfpc

        for ch in range(n_coarse):
            f0 = fch1 + ch * nfpc * foff
            f1 = fch1 + (ch + 1) * nfpc * foff
            f_min, f_max = sorted([f0, f1])

            # Check against notch filter exclusion ranges
            skip = False
            if band in GBT_NOTCH_FILTERS:
                for lo, hi in GBT_NOTCH_FILTERS[band]:
                    if lo <= f_min <= hi or lo <= f_max <= hi:
                        skip = True
                        break
            if not skip:
                tasks.append((h5, ch))

    return tasks


def split_tasks(tasks, train_frac=0.8, seed=42):
    """
    Shuffle & split the flat task list into train vs. val sets.
    Returns two sets of (h5_path, channel_idx).
    """
    random.seed(seed)
    shuffled = tasks.copy()
    random.shuffle(shuffled)
    cut = int(train_frac * len(shuffled))
    train = set(shuffled[:cut])
    val   = set(shuffled[cut:])
    return train, val

In [ ]:
def boxes_intersect(boxA, boxB, min_overlap_frac=0.0):
    """
    Returns True if boxA and boxB (cx,cy,w,h normalized) overlap by at least
    min_overlap_frac of the smaller box’s area.  If min_overlap_frac=0, any positive
    intersection counts.
    """
    cxA, cyA, wA, hA = boxA
    cxB, cyB, wB, hB = boxB

    # convert to (x1,y1,x2,y2)
    x1A, y1A = cxA - wA/2, cyA - hA/2
    x2A, y2A = cxA + wA/2, cyA + hA/2
    x1B, y1B = cxB - wB/2, cyB - hB/2
    x2B, y2B = cxB + wB/2, cyB + hB/2

    # find intersection rectangle
    xi1, yi1 = max(x1A, x1B), max(y1A, y1B)
    xi2, yi2 = min(x2A, x2B), min(y2A, y2B)

    inter_w = max(0.0, xi2 - xi1)
    inter_h = max(0.0, yi2 - yi1)
    inter_area = inter_w * inter_h
    if inter_area <= 0:
        return False

    if min_overlap_frac <= 0:
        return True

    # else require intersection >= fraction of smaller box
    areaA = wA * hA
    areaB = wB * hB
    smaller = min(areaA, areaB)
    return (inter_area / smaller) >= min_overlap_frac


def merge_overlaps_by_coords(boxes, overlap_frac=0.0):
    """
    Group boxes that intersect (by at least overlap_frac of the smaller),
    then merge each group into one tight rectangle.
    """
    clusters = []
    used = set()

    for i, b in enumerate(boxes):
        if i in used:
            continue
        cluster = [b]
        used.add(i)
        changed = True
        while changed:
            changed = False
            for j, bj in enumerate(boxes):
                if j in used:
                    continue
                if any(boxes_intersect(bj, ck, overlap_frac) for ck in cluster):
                    cluster.append(bj)
                    used.add(j)
                    changed = True
        clusters.append(cluster)

    merged = []
    for cl in clusters:
        # to corner coords
        rects = []
        for cx, cy, w, h in cl:
            x1, y1 = cx - w/2, cy - h/2
            x2, y2 = cx + w/2, cy + h/2
            rects.append((x1, y1, x2, y2))
        x1 = min(r[0] for r in rects)
        y1 = min(r[1] for r in rects)
        x2 = max(r[2] for r in rects)
        y2 = max(r[3] for r in rects)
        merged.append(((x1+x2)/2, (y1+y2)/2, x2-x1, y2-y1))
    return merged


In [ ]:
import cv2
import numpy as np
import torch
from torch import nn
from pathlib import Path
from PIL import Image, ImageDraw
import blimpy as bl
from skimage.segmentation import slic
from skimage.transform import probabilistic_hough_line
import gc
import traceback
from scipy.stats import kurtosis
import imageio
#from yolo_moe_pytorch_channel import generate_yolo_boxes

# ─── 1) Define your rule-based gate ──────────────────────────────────────
class RuleGate(nn.Module):
    def __init__(self, in_feats=3, hidden=16):
        super().__init__()
        # in_feats = number of heuristics per map (e.g. 3: density, alignment, linearity)
        # We’ll concatenate all four maps’ heuristics → 4*in_feats inputs
        self.net = nn.Sequential(
            nn.Linear(4*in_feats, hidden),
            nn.ReLU(inplace=True),
            nn.Linear(hidden, 4)      # one logit per detector
        )

    def forward(self,
                h_sf:    torch.Tensor,  # (B,in_feats)
                h_ued:   torch.Tensor,  # (B,in_feats)
                h_canny: torch.Tensor,  # (B,in_feats)
                h_hough: torch.Tensor   # (B,in_feats)
               ):
        # 1) concatenate all heuristics
        x = torch.cat([h_sf, h_ued, h_canny, h_hough], dim=1)      # (B,4*in_feats)
        logits = self.net(x)                                        # (B,4)
        weights = torch.softmax(logits, dim=1)                      # (B,4) sums to 1
        return weights[:,0], weights[:,1], weights[:,2], weights[:,3]

# ─── 2) Heuristic helper (no change) ────────────────────────────────────
def compute_heuristics(edge_map, gray):
    mask = edge_map > 0.2
    density = float(mask.mean())
    # alignment
    if density > 0:
        gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, 3)
        gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, 3)
        alignment = float(np.hypot(gx, gy)[mask].mean())
    else:
        alignment = 0.0
    # entropy
    # entropy = float(-np.sum(edge_map * np.log2(edge_map + 1e-8)))
    # linearity (aspect‐ratio proxy)
    bw = mask.astype(np.uint8)
    n, labels = cv2.connectedComponents(bw)
    ratios = []
    for L in range(1, n):
        ys, xs = np.where(labels == L)
        if ys.size:
            h, w = np.ptp(ys)+1, np.ptp(xs)+1
            ratios.append(float(w)/h if h else 0.0)
    linearity = float(np.mean(ratios)) if ratios else 0.0

    if density < 0.01:
        return [0.0, 0.0, 0.0]
    return [density, alignment, linearity]


# ─── 3) The new process_file ─────────────────────────────────────────────
def process_file(job):
    (h5_path, channels, gidxs,
     base_dir, class_id, train_set, val_set,
     pad_width) = job

    # Load .h5 once
    fb   = bl.Waterfall(h5_path, load_data=True)
    data = 10*np.log10(fb.data.squeeze())  # (ntime, nfreq)


    # Prepare SF detector and gate
    EDGE_MODEL = "/home/jliang/gbt-rfi/model.yml.gz"
    sf_det     = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL)
    # gate       = RuleGate()  # use default w_align/w_ent/bias

    for ch_idx, gidx in zip(channels, gidxs):
        subset = "train" if (h5_path,ch_idx) in train_set else "val"
        img_dir = Path(base_dir)/subset/"images"
        lbl_dir = Path(base_dir)/subset/"labels"
        vis_dir = Path(base_dir)/"visualization"/subset
        for d in (img_dir,lbl_dir,vis_dir): d.mkdir(parents=True, exist_ok=True)

        # Extract & clean the block
        cw = fb.header.get("nfpc",1024)
        f0, f1 = ch_idx*cw, (ch_idx+1)*cw
        block = data[:,f0:f1]
        low, high = int(0.15*cw), int(0.85*cw)
        block = block[:,low:high]
        h_img, w_img = block.shape

        # Remove vertical line artifact
        rows, cols = block.shape
        # 1) Define a little window around the true center column
        mid = cols // 2
        half_win = 2                 # look 2 columns on either side
        start = max(mid - half_win, 0)
        end   = min(mid + half_win + 1, cols)

        # 2) Compute the mean of each of those columns
        center_window = block[:, start:end]            # shape = (rows, window_width)
        vert_means    = center_window.mean(axis=0)     # length = window_width

        # 3) Locate the spike RELATIVE to the window, then map back
        rel_idx    = np.argmax(vert_means)             # in [0 .. window_width-1]
        center_col = start + rel_idx                   # actual column index in block

        # 4) Replace that column by the average of its two neighbors (if they exist)
        left_col  = center_col - 1
        right_col = center_col + 1

        if 0 <= left_col < cols and 0 <= right_col < cols:
            block[:, center_col] = 0.5*(block[:, left_col] + block[:, right_col])
        elif 0 <= left_col < cols:
            block[:, center_col] = block[:, left_col]
        elif 0 <= right_col < cols:
            block[:, center_col] = block[:, right_col]

        # Build img for SF
        norm = (block - block.min())/(np.ptp(block)+1e-6)
        img3 = np.stack([norm]*3, axis=-1).astype(np.float32)
        sf_map = sf_det.detectEdges(img3).squeeze()
        gray8 = (255*norm).astype(np.uint8)
        kurt_val = kurtosis(gray8.flatten(), fisher=False)
        if kurt_val < 7.0:
            continue
        # Build filename
        f_start = fb.header['fch1'] + f0 * fb.header['foff']
        f_stop  = fb.header['fch1'] + (f1 - 1) * fb.header['foff']
        fn = f"img_{kurt_val:0{pad_width}.2f}_f_{f_start:.4f}_{f_stop:.4f}.png"
        if (img_dir/fn).exists():
            continue

        # Sobel map
        gx = cv2.Sobel(gray8, cv2.CV_32F, 1,0,3)
        gy = cv2.Sobel(gray8, cv2.CV_32F, 0,1,3)
        ued_map = np.hypot(gx, gy)
        ued_map = cv2.normalize(ued_map, None, 0,1, cv2.NORM_MINMAX)

        # Canny
        low_thresh  = 50
        high_thresh = 150
        gray_denoised = cv2.medianBlur(gray8, 5)   # or cv2.GaussianBlur(gray8, (5,5), 0)
        edges = cv2.Canny(gray_denoised, low_thresh, high_thresh)
        contours, _ = cv2.findContours(edges,
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)
        # after findContours…
        contour_mask = np.zeros_like(edges, dtype=np.uint8)
        cv2.drawContours(contour_mask, contours, -1, 1, thickness=-1)

        # and use contour_mask directly as your “canny_map”
        canny_map = contour_mask.astype(np.uint8)


        # Hough
        hough_map = np.zeros_like(canny_map, dtype=np.float32)
        lines = probabilistic_hough_line(
            (edges > 0).astype(np.uint8),
            threshold=5,
            line_length=10,
            line_gap=2
        )
        for (y0, x0), (y1, x1) in lines:
            cv2.line(hough_map, (x0, y0), (x1, y1), 1.0, 1)
        hough_map = cv2.GaussianBlur(hough_map, (3,3), 0)

        # 1) Compute global heuristics on full maps
        # h_sf    = compute_heuristics(sf_map,    gray8)
        # h_ued   = compute_heuristics(ued_map,   gray8)
        # h_canny = compute_heuristics(canny_map, gray8)
        # h_hough = compute_heuristics(hough_map, gray8)

        # # 2) Run through your gate (batch size = 1)
        # h_sf_t    = torch.tensor([h_sf],    dtype=torch.float32)
        # h_ued_t   = torch.tensor([h_ued],   dtype=torch.float32)
        # h_canny_t = torch.tensor([h_canny], dtype=torch.float32)
        # h_hough_t = torch.tensor([h_hough], dtype=torch.float32)
        # w_sf, w_ued, w_canny, w_hough = gate(h_sf_t, h_ued_t, h_canny_t, h_hough_t)
        # w_sf, w_ued, w_canny, w_hough = [w.item() for w in (w_sf, w_ued, w_canny, w_hough)]
        w_sf, w_ued, w_canny, w_hough = [0.25, 0.25, 0.25, 0.25]

        # 3) Fuse whole-image maps with those weights
        fused_global = (
            w_sf    * sf_map +
            w_ued   * ued_map +
            w_canny * canny_map +
            w_hough * hough_map
        )

        # === Z-score normalize the full map ===
        mean = fused_global.mean()
        std = fused_global.std()
        zmap = (fused_global - mean) / (std + 1e-8)

        # === Then threshold on zmap, e.g. ===
        binary = (zmap > 2.0)

        lbl, num = ndi.label(binary)
        sizes = np.bincount(lbl.ravel())
        clean = np.zeros_like(binary)
        for i, sz in enumerate(sizes):
            if i>0 and sz>=50:      # just drop very tiny blobs
                clean[lbl==i] = True

        # 1.2 Bridge very nearby regions (so that close boxes will fuse)
        #    a dilation with a 20×20 square will join components separated by ≤2 pixels
        k = 20
        # a horizontal line structuring element (bridges gaps up to k pixels left/right)
        se_h = np.ones((1, k), bool)

        # a vertical line structuring element (bridges gaps up to k pixels up/down)
        se_v = np.ones((k, 1), bool)

        # dilate separately in each orientation…
        bridged_h = ndi.binary_dilation(clean, structure=se_h)
        bridged_v = ndi.binary_dilation(clean, structure=se_v)

        # …then take their union
        bridged = bridged_h | bridged_v
        # 1.3 (Optional) re-fill small holes in the newly merged blobs
        filled = ndi.binary_closing(bridged, structure=np.ones((3,3),bool))


        # YOLO boxes
        boxes = generate_yolo_boxes(filled)
        if boxes is None:
            boxes = []
        # Merge overlapping boxes
        boxes = merge_overlaps_by_coords(boxes)
        # compute global mean brightness once
        global_mean = block.mean()

        # choose how much darker to allow (e.g. 10% darker)
        tol = 0.10

        good_boxes = []
        for cx, cy, w_n, h_n in boxes:
            # map back to pixel coords
            x0 = int((cx - w_n/2) * w_img)
            x1 = int((cx + w_n/2) * w_img)
            y0 = int((cy - h_n/2) * h_img)
            y1 = int((cy + h_n/2) * h_img)

            # clamp
            x0, x1 = max(0, x0), min(w_img, x1)
            y0, y1 = max(0, y0), min(h_img, y1)

            patch = block[y0:y1, x0:x1]
            if patch.size == 0:
                continue

            patch_mean = patch.mean()
            # only keep it if it’s no more than tol below global
            if patch_mean >= (1 - tol) * global_mean:
                good_boxes.append((cx, cy, w_n, h_n))

        boxes = good_boxes

        del sf_map, ued_map, canny_map, hough_map
        del fused_global, zmap
        gc.collect()
        # [.. save PNG, write .txt, draw visualization exactly as before ..]
        # Calculate frequency range of this coarse channel
        f_start = fb.header['fch1'] + f0 * fb.header['foff']
        f_stop  = fb.header['fch1'] + (f1 - 1) * fb.header['foff']
        
        # Build filename
        f_start = fb.header['fch1'] + f0 * fb.header['foff']
        f_stop  = fb.header['fch1'] + (f1 - 1) * fb.header['foff']
        kurt_val = kurtosis(gray8.flatten(), fisher=False)
        if kurt_val < 7.0:
            continue  # skip if kurtosis is too low (e.g. noise)
        fn = f"img_{kurt_val:0{pad_width}.3f}_f_{f_start:.4f}_{f_stop:.4f}.png"
        img_path = img_dir / fn
        txt_path = lbl_dir / fn.replace(".png", ".txt")

        if len(boxes) == 0:
            continue  # skip if no boxes found

        # Save image
        arr8 = (255 * (block - block.min()) / (np.ptp(block) + 1e-6)).astype(np.uint8)
        img = Image.fromarray(np.stack([arr8]*3, axis=-1))
        img.save(img_path)

        # Write YOLO labels
        with open(txt_path, "w") as f:
            for x_c, y_c, w_n, h_n in boxes:
                f.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}\n")

        # # If empty, log it
        # if not boxes:
        #     (lbl_dir/"empty_labels.csv").open("a").write(f"{fn},{h5_path},{ch_idx}\n")

        # Draw & save visualization
        vis_img = img.convert("RGB")
        draw    = ImageDraw.Draw(vis_img)
        for x_c, y_c, w_n, h_n in boxes:
            xc, yc = x_c*w_img, y_c*h_img
            bw, bh = w_n*w_img, h_n*h_img
            x0, y0 = int(xc - bw/2), int(yc - bh/2)
            x1, y1 = int(xc + bw/2), int(yc + bh/2)
            draw.rectangle([x0, y0, x1, y1], outline="red", width=2)
        vis_img.save(vis_dir/fn)
        # debug_dir = vis_dir/"debug_masks"
        # debug_dir.mkdir(exist_ok=True)
        # imageio.imwrite(debug_dir/f"binary_{kurt_val}_{f_start}_{f_stop}.png",   (binary*255).astype(np.uint8))
        del binary
    del data, fb, h5_path, channels, gidxs
    gc.collect()

if __name__ == "__main__":
    tasks = build_tasks(df)
    train_set, val_set = split_tasks(tasks)

    # 2) build job tuples of exactly what process_file unpacks:
    job_args = []
    BASE_DIR = "/datax/scratch/jliang/dataset_final_7"
    NUM_WORKERS = 2
    class_id = 0  # Assuming a single class for simplicity
    pad_width = 4  # Zero-padding width for gidx in filenames
    for idx, (h5_path, ch_idx) in enumerate(tasks):
        job = (
            h5_path,
            [ch_idx],        # channels
            [idx],           # gidxs
            BASE_DIR,
            class_id,
            train_set,
            val_set,
            pad_width
        )
        job_args.append(job)

    # 3) submit each job as a single argument
    with ProcessPoolExecutor(max_workers=NUM_WORKERS) as exe:
        futures = {
            exe.submit(process_file, job): (job[0], job[1])
            for job in job_args
        }

        for future in tqdm(as_completed(futures), total=len(futures), ascii=True):
            h5, ch_list = futures[future]
            try:
                future.result()
            except Exception as e:
                print(f"Error processing {h5} channel {ch_list}: {e}")
                traceback.print_exc()
                raise

